# OpenAI Gym

As we saw in the previous chapter, there are several entities in RL's view of the world:
Agent: A person or a thing that takes an active role. In practice, it's some piece of code, which implements some policy. Basically, this policy must decide what action is needed at every time step, given our observations.

Environment: Some model of the world, which is external to the agent and has the responsibility of providing us with observations and giving us rewards. It changes its state based on our actions.

Gym is a toolkit for developing and comparing reinforcement learning algorithms. It supports teaching agents everything from walking to playing games like Pong or Pinball. The main goal of Gym is to provide a rich collection of environments for RL experiments using a unified interface.

In [1]:
import random


class Environment:
    def __init__(self):
        self.steps_left = 10

    def get_observation(self):
        return [0.0, 0.0, 0.0]

    def get_actions(self):
        return [0, 1]

    def is_done(self):
        return self.steps_left == 0

    def action(self, action):
        if self.is_done():
            raise Exception("Game is over")
        self.steps_left -= 1
        return random.random()


class Agent:
    def __init__(self):
        self.total_reward = 0.0

    def step(self, env):
        current_obs = env.get_observation()
        actions = env.get_actions()
        reward = env.action(random.choice(actions))
        self.total_reward += reward


if __name__ == "__main__":
    env = Environment()
    agent = Agent()

    while not env.is_done():
        agent.step(env)

    print("Total reward got: %.4f" % agent.total_reward)

Total reward got: 4.3827


From high level, every environment provides you with these pieces of information and functionality:
    
A set of actions that are allowed to be executed in an environment. Gym supports both discrete and continuous actions, as well as their combination. <br>

The shape and boundaries of the observations that an environment provides the agent with.

A method called step to execute an action, which returns the current observation, reward, and indication that the episode is over.

A method called reset to return the environment to its initial state and to obtain the first observation. <br>

### Action space
As you may remember, the actions that an agent can execute can be discrete, continuous, or a combination of both. Discrete actions are a fixed set of things that an agent could do, for example, directions in a grid like left, right, up, or down. Another example is a push button, which could be either pressed or released. Both states are mutually exclusive, because a main characteristic of a discrete action space is that only one action from the action space is possible.<br>
A continuous action has a value attached to it, for instance, a steering wheel, which can be turned at a specific angle, or an accelerator pedal, which can be pressed with different levels of force. A description of a continuous action includes the boundaries of the value that the action could have.<br>

### Observation space
Observations are pieces of information that an environment provides the agent with, on every timestamp, besides the reward. Observations can be as simple as a bunch of numbers or as complex as several multidimensional tensors containing color images from several cameras. An observation can even be discrete, much like action spaces.

### The environment
The environment is represented in Gym by the Env class, which has the following members:
action_space: This is the field of the Space class, providing a specification for allowed actions in the environment.<br>

observation_space: This field has the same Space class, but specifies the observations provided by the environment.<br>

reset(): This resets the environment to its initial state, returning the initial observation vector <br>

step(): This method allows the agent to give the action and returns the information about the outcome of the 

action: the next observation, local reward, and end-of-episode flag. This method is a bit complicated and we'll look at it in detail later in this section.<br>

Gym 0.9.3 comes with an impressive list of 116 unique environments, which can be divided into several groups:<br>

Classic control problems: These are toy tasks that are used in optimal control theory and RL papers as benchmarks or demonstrations. They are usually simple, with a low-dimension observation and action spaces, but they are useful as quick checks when implementing algorithms. Think about them as the "MNIST for RL" (in case you haven't heard about MNIST, it is a handwriting digit recognition dataset from Yann LeCun).<br>

Atari 2600: These are games from the classic game platform from the 1970s. There are 63 unique games.<br>

Algorithmic: These are problems that aim to perform small computation tasks, such as copying the observed sequence or adding numbers.<br>

Board games: These are the games of Go and Hex.<br>

Box2D: These are environments that use the Box2D physics simulator to learn walking or car control.<br>

MuJoCo: This is another physics simulator used for several continuous control problems.<br>

Parameter tuning: This is RL being used to optimize neural network parameters.<r>

Toy text: These are simple grid-world text environments.<br>

PyGame: These are several environments implemented using the PyGame engine.<br>

Doom: These are nine mini-games implemented on top of ViZdoom.<br>

### Catrpole example

Here we will import the Gym package and create an environment called CartPole. This environment is from the "classic control" group and its gist is to control the platform with a stick attached by its bottom part (see the following figure). The trickiness is that this stick tends to fall right or left and you need to balance it by moving the platform to the right or left on every step.

The observation of this environment is four float numbers containing information about the x coordinate of the stick's center of mass, its speed, its angle to the platform, and its angular speed. Of course, by applying some math and physics knowledge, it won't be complicated to convert these numbers into actions when we need to balance the stick, but our problem is much trickier: how do we learn to balance this system without knowing the exact meaning of the observed numbers and only by getting the reward? The reward in this environment is 1 given on every time step. The episode continues until the stick falls; so to get a more accumulated reward, we need to balance the platform in a way to avoid the stick falling.

![Fig](imgs/img_001.png)

In [2]:
import gym


if __name__ == "__main__":
    env = gym.make("CartPole-v0")

    total_reward = 0.0
    total_steps = 0
    obs = env.reset()

    while True:
        action = env.action_space.sample()
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        total_steps += 1
        if done:
            break

    print("Episode done in %d steps, total reward %.2f" % (total_steps, total_reward))


Episode done in 30 steps, total reward 30.00


### Wrappers

Very frequently, you will want to extend the environment's functionality in some generic way. For example, an environment gives you some observations, but you want to accumulate them in some buffer and provide to the agent the N last observations, which is a common scenario for dynamic computer games, when one single frame is just not enough to get the full information about the game state. Another example is when you want to be able to crop or preprocess an image's pixels to make it more convenient for the agent to digest or if you want to normalize reward scores somehow. There are many such situations that have the same structure: you'd like to "wrap" the existing environment and add some extra logic doing something. Gym provides you with a convenient framework for these situations, called the Wrapper class. The class structure is shown in the following diagram.

The Wrapper class inherits the Env class. Its constructor accepts the only argument: the instance of the Env class to be "wrapped." To add extra functionality, you need to redefine the methods you want to extend such as step() or reset(). 

![Fig](imgs/img_002.png)

In [3]:
import gym

import random


class RandomActionWrapper(gym.ActionWrapper):
    def __init__(self, env, epsilon=0.1):
        super(RandomActionWrapper, self).__init__(env)
        self.epsilon = epsilon

#     This is a method that we need to override from a parent's class to tweak the agent's actions. 
#     Every time we roll the die and with the probability of epsilon, we sample a random action 
#     from the action space and return it instead of the action the agent has sent to us. 
#     Note that using action_space and wrapper abstractions, we were able to write abstract code, 
#     which will work with any environment from the Gym.
    def action(self, action):
        if random.random() < self.epsilon:
            print("Random!")
            return self.env.action_space.sample()
        return action


if __name__ == "__main__":
    env = RandomActionWrapper(gym.make("CartPole-v0"))

    obs = env.reset()
    total_reward = 0.0

    while True:
        obs, reward, done, _ = env.step(0)
        total_reward += reward
        if done:
            break

    print("Reward got: %.2f" % total_reward)

Random!
Random!
Reward got: 10.00


### Cartpole Random Monitor

Another class you should be aware of is Monitor. It is implemented like Wrapper and can write information about your agent's performance in a file with an optional video recording of your agent in action. Some time ago, it was possible to upload the result of the Monitor class' recording to the https://gym.openai.com website and see your agent's position in comparison to other people's results (see thee following screenshot), but, unfortunately, at the end of August 2017, OpenAI decided to shut down this upload functionality and froze all the results. There are several activities to implement an alternative to the original website, but they are not ready yet.

In [8]:
import gym


if __name__ == "__main__":
    env = gym.make("CartPole-v0")
    env = gym.wrappers.Monitor(env, "recording", force=True)

    total_reward = 0.0
    total_steps = 0
    obs = env.reset()

    while True:
        action = env.action_space.sample()
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        total_steps += 1
        if done:
            break

    print("Episode done in %d steps, total reward %.2f" % (total_steps, total_reward))
    env.close()
    env.env.close()

Episode done in 9 steps, total reward 9.00


The Monitor class requires the FFmpeg utility to be present on the system, which is used to convert captured observations into an output video file. This utility must be available, otherwise Monitor will raise an exception. The easiest way to install FFmpeg is using your system's package manager, which is OS distribution-specific.
To start this example, one of these three extra prerequisites should be met:<br>
The code should be run in an X11 session with the OpenGL extension (GLX)<br>
The code should be started in an Xvfb virtual display<br>
You can use X11 forwarding in ssh connection <br>
The cause of this is video recording, which is done by taking screenshots of the window drawn by the environment. <br>

Some of the environment uses OpenGL to draw its picture, so the graphical mode with OpenGL needs to be present. This could be a problem for a virtual machine in the cloud, which physically doesn't have a monitor and graphical interface running. To overcome this, there is a special "virtual" graphical display, called Xvfb (X11 virtual framebuffer), which basically starts a virtual graphical display on the server and forces the program to draw inside it. This would be enough to make Monitor happily create the desired videos.<br>